In [63]:
import cv2
import os
import torch
import numpy as np

# Write the function for uploading all png files in the file folder.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

folder = r"E:\archive\images_001\images"

# I realize that png files are very large. Thus, I only select 81 images in images_001 as the training set to build the naive model on my laptop. 
# They are images for patient No. 32, 72, 147, 149 and 150.

In [64]:
img = load_images_from_folder(folder)

In [ ]:
img_tensor = torch.FloatTensor(img)

In [66]:
(n, height, width, color_depth) = np.shape(img) # 81*1024*1024*3

In [67]:
# According to BBox_List_2017, it seems that it contains all images which show there is some disease.

# This is a two-class classification problem. I use the probability of disease as the output. 
# To avoid NaN, I use 0.01 for non-disease and 0.99 for disease in the training set
result = 0.01 + np.zeros((n,1), dtype=float) 

I do have questions about the kinds of diseases. Is this a 2-class classification or multi-class classification? It seems that in README_CHESTXRAY.pdf there are many kinds of lung disease shown in X-ray images.

In [68]:
# Among these 81 images, there are 32_037,72_000 ,147_001, 149_006 and 150_002 which show the disease.
result[37] = result[37] + 0.98
result[61] = result[61] + 0.98
result[63] = result[63] + 0.98
result[71] = result[71] + 0.98
result[77] = result[77] + 0.98
result_tensor = torch.FloatTensor(result)

In [108]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# B is batch size
B = 50

# H is hidden dimension
H = 30

# Create input and output data
x = img_tensor
x = torch.reshape(x, (n, height*width*color_depth))
y = result_tensor # two class classification

# Randomly initialize weights
w1 = torch.randn(height*width*color_depth, H, device = device, dtype = dtype)
w2 = torch.randn(H, 1, device = device, dtype = dtype)

learning_rate = 1e-6
for t in range(10):
    # Forward pass: compute predicted y
    # y_hat = h_2 *w_2 = (h - 2)*w_2 = (x * w_1 - 2)*w_2
    h = torch.mm(x, w1) - 2
    h_2 = torch.mm(h, w2)
    y_pred = torch.sigmoid(h_2)
    y_pred[y_pred < 0.01 ] = 0.01
    y_pred[y_pred > 0.99 ] = 0.99 # To avoid NaN
    
    # Compute and print cross-entropy loss
    # L(y_hat; w) = -y*log(1-y_hat) - (1-y)*log(1-y_hat)
    loss = (-y*np.log(y_pred)-(1-y)*np.log(1-y_pred)).sum()
    print(t, loss)

    # Backward propogation to compute gradients of w1 and w2 with respect to loss
    # dL/dy_hat = -y/(y_hat*(1-y_hat))
    grad_y_pred = -y/(y_pred*(1-y_pred))
    # dy_hat/dh_2 = y_hat*(1-y_hat)
    grad_h2 = y_pred*(1-y_pred)
    # dy_hat/dw_2 = h
    grad_w2 = torch.mm(torch.t(h), -y)
    # dy_hat/dh = w_2
    grad_h = torch.mm(-y, torch.t(w2))
    # dh/dw_1 = x
    grad_w1 = torch.mm(torch.t(x), grad_h)

    # Update weights using gradient descent
    w1 = w1 - learning_rate * grad_w1
    w2 = w2 - learning_rate * grad_w2

0 tensor(283.7357)
1 tensor(346.7810)
2 tensor(346.7810)
3 tensor(346.7810)
4 tensor(346.7810)
5 tensor(346.7810)
6 tensor(346.7810)
7 tensor(346.7810)
8 tensor(346.7810)
9 tensor(346.7810)


In [110]:
#  Similarly, I only select 81 images in images_002 as the test set. 
# They are images for patient No. 1369 and 1373-1378.

test_folder=r"E:\archive\images_002\images"

img_test = load_images_from_folder(test_folder)

img_tensor_test = torch.FloatTensor(img_test)

(n_test, height, width, color_depth) = np.shape(img_test)

In [111]:
# Among these 81 images, there are 1369_000, 1373_009 and 1373_039 which show the disease.
# To avoid NaN, I use 0.01 for non-disease and 0.99 for disease in the training set
test_result = 0.01 + np.zeros((n_test,1), dtype=float)
test_result[0] = test_result[0] + 0.98
test_result[10] = test_result[10] + 0.98
test_result[40] = test_result[40] + 0.98
test_result_tensor = torch.FloatTensor(test_result)

In [116]:
x_test = img_tensor_test
x_test = torch.reshape(x_test, (n_test, height*width*color_depth))
y_test = test_result_tensor
h_test = torch.mm(x_test, w1) - 2
h_2_test = torch.mm(h_test, w2)
y_pred_test = torch.sigmoid(h_2_test)
y_pred_test[y_pred_test < 0.01 ] = 0.01
y_pred_test[y_pred_test > 0.99 ] = 0.99 # To avoid NaN
loss_test = (-y_test*np.log(y_pred_test)-(1-y_test)*np.log(1-y_pred_test)).sum()

In [119]:
loss_test

tensor(355.7874)

The naive model performance is bad, which is reasonable. The dataset is too small and unbalanced; the model is too simple; the number of iteration is too small. 